In [16]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import time

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cuda')

In [17]:
txt_path = '../data/tokenized/korquad_mecab.txt'
corpus = [sent.replace('\n', '').strip() for sent in open(txt_path, 'r', encoding='utf-8').readlines()]

In [18]:
corpus[0]

'1 8 3 9 년 바그너 는 괴테 의 파우스트 을 처음 읽 고 그 내용 에 마음 이 끌려 이 를 소재 로 해서 하나 의 교향곡 을 쓰 려는 뜻 을 갖 는다 . 이 시기 바그너 는 1 8 3 8 년 에 빛 독촉 으로 산전수전 을 다 걲은 상황 이 라 좌절 과 실망 에 가득 했으며 메피스토펠레스 를 만나 는 파우스트 의 심경 에 공감 했 다고 한다 . 또한 파리 에서 아 브네 크 의 지휘 로 파리 음악원 관현악단 이 연주 하 는 베토벤 의 교향곡 9 번 을 듣 고 깊 은 감명 을 받 았 는데 , 이것 이 이듬해 1 월 에 파우스트 의 서곡 으로 쓰여진 이 작품 에 조금 이 라도 영향 을 끼쳤 으리라는 것 은 의심 할 여지 가 없 다 . 여기 의 라단조 조성 의 경우 에 도 그 의 전기 에 적혀 있 는 것 처럼 단순 한 정신 적 피로 나 실의 가 반영 된 것 이 아니 라 베토벤 의 합창 교향곡 조성 의 영향 을 받 은 것 을 볼 수 있 다 . 그렇게 교향곡 작곡 을 1 8 3 9 년 부터 4 0 년 에 걸쳐 파리 에서 착수 했으나 1 악장 을 쓴 뒤 에 중단 했 다 . 또한 작품 의 완성 과 동시 에 그 는 이 서곡 ( 1 악장 ) 을 파리 음악원 의 연주회 에서 연주 할 파트 보 까지 준비 하 였으나 , 실제로 는 이루 어 지지 는 않 았 다 . 결국 초연 은 4 년 반 이 지난 후 에 드레스덴 에서 연주 되 었 고 재연 도 이루 어 졌 지만 , 이후 에 그대로 방치 되 고 말 았 다 . 그 사이 에 그 는 리엔치 와 방황 하 는 네덜란드인 을 완성 하 고 탄호이저 에 도 착수 하 는 등 분주 한 시간 을 보냈 는데 , 그런 바쁜 생활 이 이 곡 을 잊 게 한 것 이 아닌가 하 는 의견 도 있 다 .'

### 단어-문맥 행렬에 LSA 적용

In [5]:
from sklearn.decomposition import TruncatedSVD
from soynlp.vectorizer import sent_to_word_contexts_matrix

단어-문서 행렬은 각 문서에 단어가 몇 번 등장하는 지를 세어 만드는 (단어 갯수) X (문서 갯수) 크기의 행렬이다. soynlp 패키지의 sent_to_word_contexts_matrix 메소드를 사용하면 얻을 수 있다. windows는 문맥의 길이, min_tf는 최소 등장 횟수, dynamic_weight는 중심 단어와 멀 수록 가중치를 감소하는지 여부, verbose는 결과 출력 여부이다.

In [6]:
input_matrx, idx2vocab = sent_to_word_contexts_matrix(
    corpus, windows=3, min_tf=10, dynamic_weight=True, verbose=True)

Create (word, contexts) matrix
  - counting word frequency from 76945 sents, mem=0.132 Gb
  - scanning (word, context) pairs from 76945 sents, mem=0.460 Gb
  - (word, context) matrix was constructed. shape = (19127, 19127)                    
  - done


In [7]:
type(input_matrx), type(idx2vocab)

(scipy.sparse.csr.csr_matrix, list)

In [8]:
input_matrx.shape

(19127, 19127)

In [9]:
%%time
cooc_svd = TruncatedSVD(n_components=100)
cooc_vecs = cooc_svd.fit_transform(input_matrx)

CPU times: user 13 s, sys: 8.35 s, total: 21.3 s
Wall time: 3.66 s


In [10]:
cooc_vecs.shape

(19127, 100)

In [12]:
word2id = dict()
for word in idx2vocab:
    word2id[word]=len(word2id)

In [58]:
eps =1e-6
def cos_sim(a,b):
    return np.sum((a*b)/(np.linalg.norm(a)*np.linalg.norm(b)+eps),1)

a = cooc_vecs[word2id['집']]
b = cooc_vecs
cos_mat = cos_sim(a,b)
sim, indices = torch.topk(torch.tensor(cos_mat),6)
sim, indices

(tensor([0.1280, 0.1027, 0.0988, 0.0978, 0.0924, 0.0876], dtype=torch.float64),
 tensor([ 1, 12,  4,  8,  0,  3]))

In [66]:
idx2vocab[7]

'하'

In [59]:
def similar_words(mat, word, k=10):
    eps =1e-6
    def cos_sim(a,b):
        return np.sum((a*b)/(np.linalg.norm(a)*np.linalg.norm(b)+eps),1)

    word_vec = mat[word2id[word]]
    
    cos_mat = cos_sim(word_vec, mat)
    sim, indices = torch.topk(torch.tensor(cos_mat),k+1)
    
    
    word_list = []
    for i in indices:
        if i != word2id[word]:
            word_list.append(idx2vocab[i])
    return word_list


In [61]:
similar_words(cooc_vecs,'집')#.shape

['는', '0', '은', '1', '의', '이', '을', '.', '?', '다', '하']

### PPMI 행렬에 LSA 적용

점별 상호 정보량(PMI)는 다음과 같이 정의된다. 

$$ PMI(A,B) = \log \frac{P(A,B)}{P(A)\times P(B)} $$

분자가 문모보다 작을 경우 PMI는 음수가 되는데, 이는 A, B 두 단어가 동시에 등장할 확률이 두 단어가 독립일 때의 확률보다 작을 때 발생한다. 이러한 수치는 말뭉치의 크기가 충분히 크지 않은 한 신뢰하기가 어렵다(음수가 나오는 게 말이 안된다는 뜻). 

따라서 NLP에서는 PMI 대신 양의 점별 상호 정보량(PPMI)을 사용한다.

$$ PPMI(A,B) = \max (PMI(A,B),0) $$

그리고 Shifted PMI란 PMI에서 $\log{k}$를 빼준 값이다.

$$ SPMI(A,B) = PMI(A,B) - \log{k} $$

In [21]:
from soynlp.word import pmi
ppmi_matrix, _, _ = pmi(input_matrx, min_pmi=0) # 0보다 작은 값은 0으로 치환하므로 PPMI 행렬을 얻을 수 있음
ppmi_svd = TruncatedSVD(n_components=100)
ppmi_vecs = ppmi_svd.fit_transform(input_matrx)

각 단어 벡터의 유사도 기준 상위 단어 리스트 출력

In [22]:
ppmi_vecs.shape

(19127, 100)

In [38]:
a, b = ppmi_vecs[0], ppmi_vecs[1]

In [40]:
np.sum(a*b)/(np.linalg.norm(a)*np.linalg.norm(b))

0.665088516979357

In [36]:
a = [3,-4]
np.linalg.norm(a)

5.0

In [41]:
def cos_sim(a,b):
    return np.sum(a*b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [42]:
cos_sim(np.array([1,2]),np.array([1,2]))

0.9999999999999998

In [49]:
similar_words(ppmi_vecs,'한국')

KeyError: '한국'